# **Disease Detection using Symptoms and Treatment recommendation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# importing nltk to download resources for stopwords and wordnet
import nltk
nltk.download('all')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
pip install -U deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 8.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 4.7 MB/s 


In [ ]:
# importing all libraries
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split,cross_val_score
import math
import operator
import pickle
import re
#from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from itertools import combinations
from time import time
from collections import Counter
import operator
import warnings
import json
from google.colab import files
src = list(files.upload().values())[0]
#open('Treatment.py','wb').write(src)
from Treatment import diseaseDetail
uploaded = files.upload()
# ignore warnings generated due to usage of old version of tensorflow
warnings.simplefilter("ignore")
from deep_translator import GoogleTranslator

Saving Treatment.py to Treatment.py


Saving dict.json to dict.json


**Disease Symptom dataset** was created in a separate python program.

**Dataset scrapping** was done using **NHP website** and **wikipedia data**

In [ ]:
# Load Dataset scraped from NHP (https://www.nhp.gov.in/disease-a-z) & Wikipedia
# Scrapping and creation of dataset csv is done in a separate program
df=pd.read_excel("/content/drive/My Drive/MAGISTER ILMU KOMPUTER 2021/Group Stanford/Disease-Detection-based-on-Symptoms-master/Dataset/dis_sym_dataset_norm2.xlsx") # Individual Disease
documentname_list=list(df['label_dis'])
df=df.iloc[:,1:]
columns_name=list(df.columns)
documentname_list=list(documentname_list)

N=len(df)
M=len(columns_name) 

# All symptoms IDF
idf={}
for col in columns_name:
  temp=np.count_nonzero(df[col])
  idf[col]=np.log(N/temp)

# All disease,symptom TF
tf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col)
    tf[key]=df.loc[i,col]

# All disease,symptom TF.IDF
tf_idf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col) #key=(penyakit, gejala)
    tf_idf[key]=float(idf[col])*float(tf[key])

# vector of TF.IDF
D = np.zeros((N, M),dtype='float32')
for i in tf_idf:
    sym = columns_name.index(i[1])
    dis=documentname_list.index(i[0])
    D[dis][sym] = tf_idf[i]

def load(filename):	
	with open(filename) as data_file:
		data = json.load(data_file)	
  
	return data
  
# Data Synonims
mydict = load('dict.json')

def synonyms(word):
	if word in mydict.keys():
		return set(mydict[word]['sinonim'])
	else:
		return set([])

# function for cosine dot product
def cosine_dot(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0
    else:
        temp = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return temp

# convert data to lower case
def convert_tolowercase(data):
    return data.lower()

# tokenizing using regextokenizer
def regextokenizer_func(data):
    tokenizer = RegexpTokenizer(r'\w+')
    data = tokenizer.tokenize(data)
    return data

# function to generate query vector for tf_idf
def gen_vector(tokens):
    Q = np.zeros(M)
    counter = Counter(tokens)
    query_weights = {}
    for token in np.unique(tokens):
        tf = counter[token]
        try:
          idf_temp=idf[token]
        except:
          pass
        try:
            ind = columns_name.index(token)
            Q[ind] = tf*idf_temp
        except:
            pass
    return Q

# function to calculate tf_idf_score
def tf_idf_score(k, query):
    query_weights = {}
    for key in tf_idf:
        if key[1] in query:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
  
    l = []
    for i in query_weights[:k]:
        l.append(i)
    return l

# function to calculte Cosine Similarity 
def cosine_similarity(k, query):
    d_cosines = []
    query_vector = gen_vector(query)
    for d in D:
        d_cosines.append(cosine_dot(query_vector, d))
    out = np.array(d_cosines).argsort()[-k:][::-1]
  
    final_display_disease={}
    for lt in set(out):
      final_display_disease[lt] = float(d_cosines[lt])
    return final_display_disease

In [ ]:
# returns the list of synonyms of the input word from thesaurus.com (https://www.thesaurus.com/) and wordnet (https://www.nltk.org/howto/wordnet.html)
def synonyms0(term):
    synonyms = []
    #response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    #response = requests.get('https://bahasa.cs.ui.ac.id/iwn/wordnet.php'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [ ]:
# instantiate objects of libraries
splitter = RegexpTokenizer(r'\w+')

#stopwords
stop_words = stopwords.words('english')

#stemming Sastrawi
factory = StemmerFactory()
lemmatizer = factory.create_stemmer()

**Disease Symptom dataset** was created in a separate python program.

**Dataset scrapping** was done using **NHP website** and **wikipedia data**

Disease Combination dataset contains the combinations for each of the disease present in dataset as practically it is often observed that it is not necessary for a person to have a disease when all the symptoms are faced by the patient or the user.

*To tackle this problem, combinations are made with the symptoms for each disease.*

 **This increases the size of the data exponentially and helps the model to predict the disease with much better accuracy.**

*df_comb -> Dataframe consisting of dataset generated by combining symptoms for each disease.*

*df_norm -> Dataframe consisting of dataset which contains a single row for each diseases with all the symptoms for that corresponding disease.*

**Dataset contains 261 diseases and their symptoms**

In [ ]:
# Load Dataset scraped from NHP (https://www.nhp.gov.in/disease-a-z) & Wikipedia
# Scrapping and creation of dataset csv is done in a separate program
df_comb = pd.read_excel("/content/drive/My Drive/MAGISTER ILMU KOMPUTER 2021/Group Stanford/Disease-Detection-based-on-Symptoms-master/Dataset/dis_sym_dataset_comb2.xlsx") # Disease combination
df_norm = pd.read_excel("/content/drive/My Drive/MAGISTER ILMU KOMPUTER 2021/Group Stanford/Disease-Detection-based-on-Symptoms-master/Dataset/dis_sym_dataset_norm2.xlsx") # Individual Disease
Y = df_norm.iloc[:, 0:1]
X = df_norm.iloc[:, 1:]
# List of symptoms
dataset_symptoms = list(X.columns)
diseases = list(set(Y['label_dis']))
diseases.sort()

In [ ]:
dataICD = pd.read_excel("/content/drive/My Drive/MAGISTER ILMU KOMPUTER 2021/Group Stanford/Disease-Detection-based-on-Symptoms-master/Dataset/icd10.xlsx")
icd = dataICD.to_dict(orient = 'list')


In [ ]:
# Taking symptoms from user as input
# Preprocessing the input symtoms 
user_symptoms = str(input("\nMasukan gejala dipisahkan dengan koma(,):\n")).lower().split(',')
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.stem(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)  


Masukan gejala dipisahkan dengan koma(,):
panas 3 hr, batuk pilek, mata blobok, telinga kiri sakit, mata merah sdh 1 mggu, mata kiri bengkak, nyeri, gatal sejak tadi malam, pusing, mata merah, 2hr, MATA MERAH


In [ ]:
# Taking each user symptom and finding all its synonyms and appending it to the pre-processed symptom string
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
# query expansion performed by joining synonyms found for each symptoms initially entered
print("Pengembangan query dari gejala yang dimasukkan")
print(user_symptoms)

Pengembangan query dari gejala yang dimasukkan
['genting a ki erotis panas 3 hr seronok bergolak meriang kering sensual gerah bahang seksi hot  kritis demam menggairahkan a beringsang tegang n kemarau gawat kolor a berbahaya menggiurkan memberahikan merangsang n dedar', 'batuk influenza batuk darah selesma radang selaput lendir batuk pilek batu berdahak', 'preman punat bromocorah indra penglihat ain sumber bajul lubang bangsat perisau gali residivis pusat pencoleng bajingan fokus mata blobok jaharu benih bandit penjahat alat penglihat cecunguk pokok durjana punca tunas netra titik berat arah', 'ambruk kuping linu nyeri a pedih kidal mulas lara telinga kiri sakit ngilu perih sakit gering', 'preman punat bromocorah mata merah sdh 1 mggu indra penglihat ain sumber bajul lubang biram bangsat perisau sirah bangkang gali residivis pusat berma pencoleng bajingan fokus abang jaharu benih bandit penjahat ahmar alat penglihat cecunguk pokok durjana punca tunas netra titik berat arah', 'punat ben

The below procedure is performed in order to show the symptom synonmys found for the symptoms entered by the user.

The symptom synonyms and user symptoms are matched with the symptoms present in dataset. Only the symptoms which matches the symptoms present in dataset are shown back to the user. 

In [ ]:
# Loop over all the symptoms in dataset and check its similarity score to the synonym string of the user-input 
# symptoms. If similarity>0.5, add the symptom to the final list
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5: # this is jaccard similarity
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [ ]:
# Print all found symptoms
print("Gejala pencocokan teratas dari pencarian Anda!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)

# Show the related symptoms found in the dataset and ask user to select among them
select_list = input("\nSilakan pilih gejala yang relevan. Masukkan indeks (dipisahkan dengan spasi):\n").split()

# Find other relevant symptoms from the dataset based on user symptoms based on the highest co-occurance with the
# ones that is input by the user
dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])

# Symptoms that co-occur with the ones selected by user              
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   

Gejala pencocokan teratas dari pencarian Anda!
0 : sakit telinga
1 : batuk lendir berdarah
2 : batuk darah
3 : merah
4 : demam
5 : mata merah terbakar
6 : nyeri
7 : batuk lendir
8 : pusing
9 : batuk
10 : bengkak
11 : batuk termasuk batuk darah
12 : sakit
13 : sakit kepala
14 : gatal
15 : batuk berdahak
16 : mata merah


KeyboardInterrupt: ignored

In [ ]:
# Iteratively, suggest top co-occuring symptoms to the user and ask to select the ones applicable 
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nGejala umum yang terjadi bersamaan:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Apakah Anda memiliki gejala-gejala tersebut? Jika Ya, masukkan indeks (dipisahkan dengan spasi), 'tidak' untuk berhenti, '-1' untuk melewati:\n").lower().split();
        if select_list[0]=='tidak':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = []    


Gejala umum yang terjadi bersamaan:
0 : demam
1 : sakit kepala
2 : diare
3 : nyeri testis
4 : kebingungan
Apakah Anda memiliki gejala-gejala tersebut? Jika Ya, masukkan indeks (dipisahkan dengan spasi), 'tidak' untuk berhenti, '-1' untuk melewati:
tidak


Final Symptom list

In [ ]:
#Calculating TF-IDF and Cosine Similarity using matched symptoms
k = 10

print("Daftar akhir dari gejala yang diberikan untuk prediksi adalah : ")
for val in final_symp:
    print(val)

Daftar akhir dari gejala yang diberikan untuk prediksi adalah : 
biasanya tidak ada
muntah
pusing
nyeri
mual
tidak bisa bergerak
sakit perut
berkepanjangan


# **Showing the list of top k diseases to the user with their prediction probabilities.**

# **For getting information about the suggested treatments, user can enter the corresponding index to know more details.**

In [ ]:
topk1=tf_idf_score(k,final_symp)
topk2=cosine_similarity(k,final_symp)
# Show top 10 highly probable disease to the user.
print(f"\nHasil {k} teratas prediksi penyakit dengan pencocokan TF_IDF  :\n")
i = 0
topk1_index_mapping = {}
for key, score in topk1:
  print(f"{i}. Penyakit : {key} \t Score : {round(score, 2)}")
  topk1_index_mapping[i] = key
  i += 1

select = input("\nPerlu lebih banyak detail tentang penyakitnya? Masukkan indeks penyakit atau '-1' untuk menghentikan:\n")
if select!='-1':
    dis=topk1_index_mapping[int(select)]
    print()
    print(GoogleTranslator(source='auto', target='id').translate(diseaseDetail(dis)))
    print()
    print('Kode ICD 10 untuk ', dis, ' : ', icd[dis][0])


Hasil 10 teratas prediksi penyakit dengan pencocokan TF_IDF  :

0. Penyakit : Myocardial Infarction (Heart Attack) 	 Score : 8.73
1. Penyakit : Dehydration 	 Score : 7.34
2. Penyakit : Heat-Related Illnesses and Heat waves 	 Score : 7.34
3. Penyakit : Carbon monoxide poisoning 	 Score : 6.85
4. Penyakit : Anthrax 	 Score : 5.84
5. Penyakit : Calculi 	 Score : 5.84
6. Penyakit : Hepatitis A 	 Score : 5.84
7. Penyakit : Hepatitis C 	 Score : 5.56
8. Penyakit : Mad cow disease 	 Score : 5.56
9. Penyakit : Post Menopausal Bleeding 	 Score : 5.56

Perlu lebih banyak detail tentang penyakitnya? Masukkan indeks penyakit atau '-1' untuk menghentikan:
0

Infark Miokard (Serangan Jantung)
Nama lain - Infark miokard akut (AMI), serangan jantung
Spesialisasi - Kardiologi, pengobatan darurat
Gejala - Nyeri dada, sesak napas, mual, merasa ingin pingsan, keringat dingin, merasa lelah, lengan, leher, punggung, rahang, atau sakit perut
Komplikasi - Gagal jantung, detak jantung tidak teratur, syok kard

In [ ]:
# display top k diseases predicted with cosine probablity
print(f"Hasil {k} teratas prediksi penyakit dengan pencocokan Cosine Similarity :\n ")
topk2_sorted = dict(sorted(topk2.items(), key=lambda kv: kv[1], reverse=True))
j = 0
topk2_index_mapping = {}
for key in topk2_sorted:
  print(f"{j}. Penyakit : {diseases[key]} \t Score : {round(topk2_sorted[key], 2)}")
  topk2_index_mapping[j] = diseases[key]
  j += 1

    
select = input("\nPerlu lebih banyak detail tentang penyakitnya? Masukkan indeks penyakit atau '-1' untuk menghentikan dan menutup sistem:\n")
if select!='-1':
    dis=topk2_index_mapping[int(select)]
    print()
    print(GoogleTranslator(source='auto', target='id').translate(diseaseDetail(dis)))
    print()
    print('Kode ICD 10 untuk ', dis, ' : ', icd[dis][0])

Hasil 10 teratas prediksi penyakit dengan pencocokan Cosine Similarity :
 
0. Penyakit : Hepatitis C 	 Score : 0.4
1. Penyakit : Mad cow disease 	 Score : 0.24
2. Penyakit : Carbon monoxide poisoning 	 Score : 0.23
3. Penyakit : Dehydration 	 Score : 0.22
4. Penyakit : Post Menopausal Bleeding 	 Score : 0.21
5. Penyakit : Campylobacter infection 	 Score : 0.19
6. Penyakit : Myocardial Infarction (Heart Attack) 	 Score : 0.19
7. Penyakit : Sickle-cell anemia 	 Score : 0.19
8. Penyakit : Heat-Related Illnesses and Heat waves 	 Score : 0.18
9. Penyakit : Malaria 	 Score : 0.15

Perlu lebih banyak detail tentang penyakitnya? Masukkan indeks penyakit atau '-1' untuk menghentikan dan menutup sistem:
0

Hepatitis C
Spesialisasi - Gastroenterologi, Penyakit Menular
Gejala - Biasanya tidak ada
Komplikasi - Gagal hati, kanker hati, varises esofagus dan lambung
Durasi - Jangka panjang (80%)
Penyebab - Virus hepatitis C biasanya menyebar melalui kontak darah-ke-darah
Metode diagnostik - Tes darah 